In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import rioxarray 
import os

In [2]:
crops = ['Corn','Soybean', "Wheat", "Rice"]
phases = [2]

In [3]:
# Read in the data
gdf_dict = {}
for crop in crops:
    for phase in phases:
        rds = rioxarray.open_rasterio(
            "../" + os.sep + "processed" + os.sep + "output" + os.sep + crop + "_phase_" + str(phase) + "_rc_mean.asc"
        )
        rds = rds.squeeze().drop("spatial_ref").drop("band")
        rds.name = "data"
        df = rds.to_dataframe().reset_index().replace(-9, np.nan).dropna()
        # Convert to geodataframe
        gdf_dict[crop+str(phase)] = df

In [4]:
def dataframe_to_geopandas(df):
    # Convert the x and y columns to longitude and latitude, respectively
    geometry = [Point(x, y) for x, y in zip(df['x'], df['y'])]
    
    # Create a GeoDataFrame from the geometry and data columns
    gdf = gpd.GeoDataFrame(df['data'], geometry=geometry)

    # Convert to winkel tripel projection
    gdf.crs = "EPSG:4326"
    gdf = gdf.to_crs("+proj=wintri")

    return gdf

In [5]:
# Convert the dataframes to geodataframes
gdf_dict_gpd = {}
for i in gdf_dict.keys():
    gdf_dict_gpd[i] = dataframe_to_geopandas(gdf_dict[i])


In [6]:
# Read in the country borders
countries = gpd.read_file("../" + os.sep + "spatial_data" + os. sep +"countries" + os.sep + "World_Countries__Generalized_.shp")
countries.crs = "EPSG:4326"
countries = countries.to_crs('+proj=wintri')

In [10]:
# Iterate over the geodataframes and clip them to the country borders 
# And calculate the mean value of the data within the country borders
# Save mean values for the countries to a new dataframe and save it to a csv file
for crop in crops:
    for phase in phases:
        crop_gdf = gdf_dict_gpd[crop+str(phase)]
        countries_with_crops = gpd.sjoin(countries, crop_gdf, how="inner", op='intersects')
        write_header = True
        for country in set(countries_with_crops['COUNTRY']):
            country_gdf = countries_with_crops[countries_with_crops['COUNTRY'] == country]
            mean_value = country_gdf['data'].mean()
            print(country, mean_value)
            with open("../" + os.sep + "processed" + os.sep + "output" + os.sep + "country_means" + os.sep + crop + str(phase) + "mean_values.csv", "a") as f:
                if write_header:
                    f.write("country,mean_value\n")
                    write_header = False
                f.write(country + "," + str(mean_value) + "\n")



/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Uruguay -0.06397448
Poland -0.42925864
Georgia -0.001653489
Italy -0.45113698
Gabon -0.015357812
Belarus -0.3309698
Argentina -0.4228612
Portugal -0.27366188
North Korea -0.031561542
Somalia -0.054801896
Ecuador -0.107295066
Kyrgyzstan -0.1677078
Venezuela -0.25745553
China -0.2603131
Botswana 0.0
Greece -0.5355459
Sri Lanka -0.08734856
Indonesia -0.32243893
Mali -0.22933997
Bulgaria -0.37689266
Chad -0.011620763
Tajikistan -0.63405085
Turkey -0.46297437
Tanzania -0.045169134
Panama -0.009573609
Cuba -0.01653936
Eritrea -0.027926832
Cambodia -0.38662767
Canada -0.4858136
Myanmar -0.42528477
Lesotho -0.0022468648
Bhutan -0.306748
Pakistan -0.22590473
Israel -0.8351173
Australia -0.39839414
Paraguay -0.3457571
Côte d'Ivoire -0.0535618
Morocco -0.014873774
Trinidad and Tobago -0.2804461
Russian Federation -0.13221194
Benin -0.010344538
Gambia -0.013892677
Germany -0.5045182
Haiti -0.0005582778
Senegal -0.11645072
Slovakia -0.33804297
Afghanistan -0.0069841375
Rwanda -0.11500096
Azerbaijan

/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


South Korea -0.09982126
Ukraine -0.24332032
Slovenia -0.4757658
Uruguay -0.23938806
Romania -0.27016646
Italy -0.47046778
Paraguay -0.3449566
Argentina -0.31716663
Bosnia and Herzegovina -0.31742883
Thailand -0.15652375
Japan -0.043614767
Nigeria -0.0013932579
Serbia -0.43032175
Croatia -0.37113985
North Korea -0.024885168
Russian Federation -0.18152227
Ecuador -0.09682867
Benin 0.0
Slovakia -0.2009895
Bolivia -0.04438176
China -0.16270654
Vietnam -0.1018648
United States -0.381662
Rwanda -0.009587429
Brazil -0.4273134
Zimbabwe -0.06335761
Indonesia -0.035777677
Mexico -0.026399884
Albania -0.12906206
France -0.4510687
Austria -0.46825817
Zambia -0.26496938
Turkey -0.23946588
Tanzania 0.0
Moldova -0.1101239
South Africa -0.1913079
Uganda -0.003486683
Colombia -0.14637558
Iran -0.3956632
Nepal -0.1232446
Hungary -0.4556692
Laos -0.0062974147
Cambodia -0.029056469
India -0.1235055
Canada -0.3844419
Myanmar -0.08635248


/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Uruguay -0.46621126
Poland -0.51244724
Georgia -0.000825341
Italy -0.33184737
Belarus -0.33537
Argentina -0.29416096
Japan -0.16577032
Portugal -0.14389987
North Korea -0.004113271
Ecuador -0.017808009
United Kingdom -0.69481397
Algeria -0.1734125
Kyrgyzstan -0.14383116
China -0.39454883
Sudan -0.060610592
Greece -0.2958771
Bulgaria -0.44984037
Tajikistan -0.42726254
Turkey -0.35333362
Tanzania -0.02989468
Sweden -0.5772238
Jordan 0.0
Lebanon -0.50511163
Eritrea -0.08578253
Canada -0.43329933
Myanmar -0.11044377
Lesotho -0.018669302
Bhutan -0.4229138
Pakistan -0.20785168
Israel -0.19770995
Finland -0.43685687
Australia -0.34400153
Paraguay -0.17195873
Morocco -0.23370865
Russian Federation -0.17099135
Germany -0.68041223
Slovakia -0.38601023
Afghanistan -0.26180235
Rwanda -0.055375393
Azerbaijan -0.22153683
Mongolia -0.09677196
Albania -0.5471492
Spain -0.4116935
France -0.608538
Lithuania -0.3249366
Chile -0.63719183
Moldova -0.32365826
Ethiopia -0.20295304
Burundi 0.0
Colombia -0.214

/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Uruguay -0.45826715
Italy -0.3540604
Argentina -0.49140403
Japan -0.1943894
Portugal -0.41082084
North Korea -0.050203342
Ecuador -0.28372002
Liberia 0.0
Kyrgyzstan -0.11090794
Venezuela -0.27501574
China -0.39958346
Greece -0.31963572
Sri Lanka -0.19262637
Indonesia -0.20718734
Mali -0.07265715
Bulgaria -0.34108427
Chad 0.0
Tajikistan -0.18524742
Turkey -0.6222377
Tanzania -0.09956859
Panama -0.029805828
Cuba -0.06713452
Cambodia -0.13421892
Myanmar -0.2534198
Guyana -0.18568188
Bhutan -0.10412957
Pakistan -0.13373552
Australia -0.5543886
Paraguay -0.48123965
Côte d'Ivoire -0.045495566
French Guiana -0.10585835
Morocco -0.08597795
Trinidad and Tobago 0.0
Russian Federation -0.030248903
Benin -0.1922033
Gambia -0.043672383
Haiti -0.016565146
Senegal -0.30406475
Afghanistan -0.054813776
Rwanda -0.58898014
Spain -0.4393596
Chile -0.17227527
Burundi -0.12254835
Colombia -0.15817761
North Macedonia -0.40056062
Mozambique -0.0036804664
Turkmenistan 0.0
India -0.23555893
South Korea -0.22009

In [8]:
# Do a spatial join between the country borders and the crop data
countries_with_crops = gpd.sjoin(countries, crop_gdf, how="inner", op='intersects')
countries_with_crops.head()

/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Leng,SHAPE_Area,geometry,index_right,data
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3958221,-0.742757
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3958203,-0.526730
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3936608,-0.662635
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3932288,-0.732228
11,12,Guatemala,GT,Guatemala,GT,16.760150,9.193015,"POLYGON ((-9831388.759 1692437.600, -9841243.4...",3876108,-0.416150


In [9]:
for country in set(countries_with_crops['COUNTRY']):
    print(country)
    print(countries_with_crops[countries_with_crops['COUNTRY'] == country]['data'].mean())


Uruguay
-0.45826715
Italy
-0.3540604
Argentina
-0.49140403
Japan
-0.1943894
Portugal
-0.41082084
North Korea
-0.050203342
Ecuador
-0.28372002
Liberia
0.0
Kyrgyzstan
-0.11090794
Venezuela
-0.27501574
China
-0.39958346
Greece
-0.31963572
Sri Lanka
-0.19262637
Indonesia
-0.20718734
Mali
-0.07265715
Bulgaria
-0.34108427
Chad
0.0
Tajikistan
-0.18524742
Turkey
-0.6222377
Tanzania
-0.09956859
Panama
-0.029805828
Cuba
-0.06713452
Cambodia
-0.13421892
Myanmar
-0.2534198
Guyana
-0.18568188
Bhutan
-0.10412957
Pakistan
-0.13373552
Australia
-0.5543886
Paraguay
-0.48123965
Côte d'Ivoire
-0.045495566
French Guiana
-0.10585835
Morocco
-0.08597795
Trinidad and Tobago
0.0
Russian Federation
-0.030248903
Benin
-0.1922033
Gambia
-0.043672383
Haiti
-0.016565146
Senegal
-0.30406475
Afghanistan
-0.054813776
Rwanda
-0.58898014
Spain
-0.4393596
Chile
-0.17227527
Burundi
-0.12254835
Colombia
-0.15817761
North Macedonia
-0.40056062
Mozambique
-0.0036804664
Turkmenistan
0.0
India
-0.23555893
South Korea
-0.22009